# About

Here I'm trying to train _VOLO_: Vision Outlooker for Visual Recognition.  
It is a new vison transformer which achinved good results with fewer parameters than others.   

paper: https://arxiv.org/abs/2106.13112v1  
code: https://github.com/sail-sg/volo  

I uploaded official code and pretrained weights on Kaggle Datasets:  
https://www.kaggle.com/ttahara/volo-package

I make some changes for simply training binary classification task.

<br>
I have prepared the training and inference notebooks because the training time per fold is longer than resnet18d.
  
  
This is an **inference** notebook.

* model:
  * backbone: volo_d1
  * head classifier: one linear layer
  * num of input channels: **1**
* input image:
  * size: 1x256x256
  * use only on-target ('A') observations
  
  
  ```python
  img = np.load(path)[[0, 2, 4]]          # shape: (3, 273, 256)
  img = np.vstack(img)                    # shape: (819, 256)
  img = img.transpose(1, 0)               # shape: (256, 819)
  ```


* CVStrategy: Stratified KFold(K=5)

If you want to know more details of experimental settings, see training notebooks:  
[fold0](https://www.kaggle.com/ttahara/rerun-seti-e-t-volo-d1-baseline-training?scriptVersionId=68249988), [fold1](https://www.kaggle.com/ttahara/rerun-seti-e-t-volo-d1-baseline-training?scriptVersionId=68250008), [fold2](https://www.kaggle.com/ttahara/rerun-seti-e-t-volo-d1-baseline-training?scriptVersionId=68281727), [fold3](https://www.kaggle.com/ttahara/rerun-seti-e-t-volo-d1-baseline-training?scriptVersionId=68281737), [fold4](https://www.kaggle.com/ttahara/rerun-seti-e-t-volo-d1-baseline-training?scriptVersionId=68335611)

# Prapere

## Install

In [ ]:
%%bash
pip install pytorch-pfn-extras
pip install timm

## Import

In [ ]:
import os
import gc
import sys
import copy
import yaml
import random
import shutil
import typing as tp
from pathlib import Path

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.cuda import amp

import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2

import pytorch_pfn_extras as ppe
from pytorch_pfn_extras.config import Config
from pytorch_pfn_extras.training import extensions as ppe_exts, triggers as ppe_triggers

sys.path.append("../input/volo-package")
from volo.models import volo_d1, volo_d2, volo_d3, volo_d4, volo_d5  # register models to timm
from volo.utils import load_pretrained_weights as volo_load_weights

In [ ]:
ROOT = Path.cwd().parent
INPUT = ROOT / "input"
OUTPUT = ROOT / "output"
# DATA = INPUT / "seti-breakthrough-listen"
DATA = INPUT / "c" / "seti-breakthrough-listen"
TRAIN = DATA / "train"
TEST = DATA / "test"
TRAINING_OUTPUT_PATH = INPUT / "seti-breakthrough-listen-weights-for-volo-d1"

RANDAM_SEED = 1086
CLASSES = ["target",]
N_CLASSES = len(CLASSES)
FOLDS = [0, 1, 2, 3, 4]
N_FOLDS = len(FOLDS)

## Read Data, Split folds

In [ ]:
train = pd.read_csv(DATA / "train_labels.csv")
smpl_sub = pd.read_csv(DATA / "sample_submission.csv")

In [ ]:
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDAM_SEED)
train["fold"] = -1
for fold_id, (_, val_idx) in enumerate(skf.split(train["id"], train["target"])):
    train.loc[val_idx, "fold"] = fold_id

In [ ]:
train.groupby("fold").agg(total=("id", len), pos=("target", sum))

## 

## Definition

### Model

In [ ]:
VOLO_CHECHPOINTS = {
    "volo_d1": "../input/volo-package/d1_224_84.2.pth.tar",
    "volo_d1-384": "../input/volo-package/d1_384_85.2.pth.tar",
    "volo_d2": "../input/volo-package/d2_224_85.2.pth.tar",
    "volo_d2-384": "../input/volo-package/d2_384_86.0.pth.tar",
    "volo_d3": "../input/volo-package/d3_224_85.4.pth.tar",
    "volo_d3-448": "../input/volo-package/d3_448_86.3.pth.tar",
    "volo_d4": "../input/volo-package/d4_224_85.7.pth.tar",
    "volo_d4-448": "../input/volo-package/d4_448_86.79.pth.tar",
    "volo_d5": "../input/volo-package/d5_224_86.10.pth.tar",
    "volo_d5-448": "../input/volo-package/d5_448_87.0.pth.tar",
    "volo_d5-512": "../input/volo-package/d5_512_87.07.pth.tar",
}

class BasicImageModel(nn.Module):
    
    def __init__(
        self, base_name: str, dims_head: tp.List[int],
        pretrained=False, in_channels: int=3, image_size: int=224 
    ):
        """Initialize"""
        self.base_name = base_name
        super().__init__()
        model_name = base_name.split("-")[0]
        assert timm.is_model(model_name), "you can use only models in timm."
        
        if model_name[:4] == "volo":
            base_model = timm.create_model(
                model_name, img_size=image_size, mix_token=False, return_dense=False)
            in_features = base_model.head.in_features
            if pretrained:
                volo_load_weights(base_model, VOLO_CHECHPOINTS[base_name], strict=False)
            
            if in_channels != 3:
                # # change input channel
                # # I follow the manner used in timm.
                first_conv = base_model.patch_embed.conv[0]
                w_t = first_conv.weight.data  # shape: (out_ch, 3, 7, 7)
                if in_channels == 1:
                    new_w_t = w_t.sum(axis=1, keepdims=True)  # shape: (out_ch, 1, 7, 7)
                else:
                    n_repeats = (in_channels + 3 - 1) // 3
                    new_w_t = w_t.repeat((1, n_repeats, 1, 1))
                    new_w_t = new_w_t[:, :in_channels]
                    new_w_t = new_w_t * 3 / in_channels  # shape: (out_ch, in_channels, 7, 7)

                first_conv.weight.data = new_w_t
        else:
            base_model = timm.create_model(
                base_name, pretrained=pretrained, in_chans=in_channels)
            in_features = base_model.num_features
            print("load imagenet pretrained:", pretrained)
            
        base_model.reset_classifier(num_classes=0)
        self.backbone = base_model
        print(f"{base_name}: {in_features}")
        
        # # prepare head clasifier
        if dims_head[0] is None:
            dims_head[0] = in_features

        layers_list = []
        for i in range(len(dims_head) - 2):
            in_dim, out_dim = dims_head[i: i + 2]
            layers_list.extend([
                nn.Linear(in_dim, out_dim),
                nn.ReLU(), nn.Dropout(0.5),])
        layers_list.append(
            nn.Linear(dims_head[-2], dims_head[-1]))
        self.head = nn.Sequential(*layers_list)

    def forward(self, x):
        """Forward"""
        h = self.backbone(x)
        h = self.head(h)
        return h

### Dataset

In [ ]:
FilePath = tp.Union[str, Path]
Label = tp.Union[int, float, np.ndarray]


class SetiSimpleDataset(torch.utils.data.Dataset):
    """
    Dataset using 6 channels by stacking them along time-axis

    Attributes
    ----------
    paths : tp.Sequence[FilePath]
        Sequence of path to cadence snippet file
    labels : tp.Sequence[Label]
        Sequence of label for cadence snippet file
    transform: albumentations.Compose
        composed data augmentations for data
    """

    def __init__(
        self,
        paths: tp.Sequence[FilePath],
        labels: tp.Sequence[Label],
        transform: A.Compose,
    ):
        """Initialize"""
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        """Return num of cadence snippets"""
        return len(self.paths)

    def __getitem__(self, index: int):
        """Return transformed image and label for given index."""
        path, label = self.paths[index], self.labels[index]
        img = self._read_cadence_array(path)
        img = self.transform(image=img)["image"]
        return {"image": img, "target": label}

    def _read_cadence_array(self, path: Path):
        """Read cadence file and reshape"""
        img = np.load(path)  # shape: (6, 273, 256)
        img = np.vstack(img)  # shape: (1638, 256)
        img = img.transpose(1, 0)  # shape: (256, 1638)
        img = img.astype("f")[..., np.newaxis]  # shape: (256, 1638, 1)
        return img

    def lazy_init(self, paths=None, labels=None, transform=None):
        """Reset Members"""
        if paths is not None:
            self.paths = paths
        if labels is not None:
            self.labels = labels
        if transform is not None:
            self.transform = transform


class SetiAObsDataset(SetiSimpleDataset):
    """Use only on-target observation"""

    def _read_cadence_array(self, path: Path):
        """Read cadence file and reshape"""
        img = np.load(path)[[0, 2, 4]]  # shape: (3, 273, 256)
        img = np.vstack(img)  # shape: (819, 256)
        img = img.transpose(1, 0)  # shape: (256, 819)
        img = img.astype("f")[..., np.newaxis]  # shape: (256, 819, 1)
        return img

### Metric

In [ ]:
Batch = tp.Union[tp.Tuple[torch.Tensor], tp.Dict[str, torch.Tensor]]
ModelOut = tp.Union[tp.Tuple[torch.Tensor], tp.Dict[str, torch.Tensor], torch.Tensor]


class ROCAUC(nn.Module):
    """ROC AUC score"""

    def __init__(self, average="macro") -> None:
        """Initialize."""
        self.average = average
        super(ROCAUC, self).__init__()

    def forward(self, y, t) -> float:
        """Forward."""
        if isinstance(y, torch.Tensor):
            y = y.detach().cpu().numpy()
        if isinstance(t, torch.Tensor):
            t = t.detach().cpu().numpy()

        return roc_auc_score(t, y, average=self.average)


def micro_average(
    metric_func: nn.Module,
    report_name: str, prefix="val",
    pred_index: int=-1, label_index: int=-1,
    pred_key: str="logit", label_key: str="target",
) -> tp.Callable:
    """Return Metric Wrapper for Simple Mean Metric"""
    metric_sum = [0.]
    n_examples = [0]
    
    def wrapper(batch: Batch, model_output: ModelOut, is_last_batch: bool):
        """Wrapping metric function for evaluation"""
        if isinstance(batch, tuple): 
            t = batch[label_index]
        elif isinstance(batch, dict):
            t = batch[label_key]
        else:
            raise NotImplementedError

        if isinstance(model_output, tuple):
            y = model_output[pred_index]
        elif isinstance(model_output, dict):
            y = model_output[pred_key]
        else:
            y = model_output

        metric = metric_func(y, t).item()
        metric_sum[0] += metric * y.shape[0]
        n_examples[0] += y.shape[0]

        if is_last_batch:
            final_metric = metric_sum[0] / n_examples[0]
            ppe.reporting.report({f"{prefix}/{report_name}": final_metric})
            # # reset state
            metric_sum[0] = 0.
            n_examples[0] = 0

    return wrapper


def calc_across_all_batchs(
    metric_func: nn.Module,
    report_name: str, prefix="val",
    pred_index: int=-1, label_index: int=-1,
    pred_key: str="logit", label_key: str="target",
) -> tp.Callable:
    """
    Return Metric Wrapper for Metrics caluculated on all data
    
    storing predictions and labels of evry batch, finally calculating metric on them.
    """
    pred_list = []
    label_list = []
    
    def wrapper(batch: Batch, model_output: ModelOut, is_last_batch: bool):
        """Wrapping metric function for evaluation"""
        if isinstance(batch, tuple):
            t = batch[label_index]
        elif isinstance(batch, dict):
            t = batch[label_key]
        else:
            raise NotImplementedError

        if isinstance(model_output, tuple):
            y = model_output[pred_index]
        elif isinstance(model_output, dict):
            y = model_output[pred_key]
        else:
            y = model_output

        pred_list.append(y.numpy())
        label_list.append(t.numpy())

        if is_last_batch:
            pred = np.concatenate(pred_list, axis=0)
            label = np.concatenate(label_list, axis=0)
            final_metric = metric_func(pred, label)
            ppe.reporting.report({f"{prefix}/{report_name}": final_metric})
            # # reset state
            pred_list[:] = []
            label_list[:] = []

    return wrapper

### Utils

In [ ]:
def set_random_seed(seed: int = 42, deterministic: bool = False):
    """Set seeds"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = deterministic  # type: ignore


def to_device(
    tensors: tp.Union[tp.Tuple[torch.Tensor], tp.Dict[str, torch.Tensor]],
    device: torch.device, *args, **kwargs
):
    if isinstance(tensors, tuple):
        return (t.to(device, *args, **kwargs) for t in tensors)
    elif isinstance(tensors, dict):
        return {
            k: t.to(device, *args, **kwargs) for k, t in tensors.items()}
    else:
        return tensors.to(device, *args, **kwargs)

## config_types for evaluating configuration

I use [pytorch-pfn-extras](https://github.com/pfnet/pytorch-pfn-extras) for training NNs. This library has useful config systems but requires some preparation.

For more details, see [docs](https://github.com/pfnet/pytorch-pfn-extras/blob/master/docs/config.md).

In [ ]:
CONFIG_TYPES = {
    # # utils
    "__len__": lambda obj: len(obj),
    "method_call": lambda obj, method: getattr(obj, method)(),

    # # Dataset, DataLoader
    "SetiSimpleDataset": SetiSimpleDataset,
    "SetiAObsDataset": SetiAObsDataset,
    "DataLoader": torch.utils.data.DataLoader,

    # # Data Augmentation
    "Compose": A.Compose, "OneOf": A.OneOf,
    "Resize": A.Resize,
    "HorizontalFlip": A.HorizontalFlip, "VerticalFlip": A.VerticalFlip,
    "ShiftScaleRotate": A.ShiftScaleRotate,
    "RandomResizedCrop": A.RandomResizedCrop,
    "Cutout": A.Cutout,
    "ToTensorV2": ToTensorV2,

    # # Model
    "BasicImageModel": BasicImageModel,

    # # Optimizer
    "AdamW": optim.AdamW,

    # # Scheduler
    "OneCycleLR": lr_scheduler.OneCycleLR,

    # # Loss,Metric
    "BCEWithLogitsLoss": nn.BCEWithLogitsLoss,
    "ROCAUC": ROCAUC,

    # # Metric Wrapper
    "micro_average": micro_average,
    "calc_across_all_batchs": calc_across_all_batchs,

    # # PPE Extensions
    "ExtensionsManager": ppe.training.ExtensionsManager,

    "observe_lr": ppe_exts.observe_lr,
    "LogReport": ppe_exts.LogReport,
    "PlotReport": ppe_exts.PlotReport,
    "PrintReport": ppe_exts.PrintReport,
    "PrintReportNotebook": ppe_exts.PrintReportNotebook,
    "ProgressBar": ppe_exts.ProgressBar,
    "ProgressBarNotebook": ppe_exts.ProgressBarNotebook,
    "snapshot": ppe_exts.snapshot,
    "LRScheduler": ppe_exts.LRScheduler, 

    "MinValueTrigger": ppe_triggers.MinValueTrigger,
    "MaxValueTrigger": ppe_triggers.MaxValueTrigger,
    "EarlyStoppingTrigger": ppe_triggers.EarlyStoppingTrigger,
}

# Inference

In [ ]:
def get_path_label(cfg: Config, train_all: pd.DataFrame):
    """Get file path and target info."""
    use_fold = cfg["/globals/val_fold"]

    train_df = train_all[train_all["fold"] != use_fold]
    val_df = train_all[train_all["fold"] == use_fold]
    
    train_path_label = {
        "paths": [TRAIN / f"{img_id[0]}/{img_id}.npy" for img_id in train_df["id"].values],
        "labels": train_df[CLASSES].values.astype("f")}
    val_path_label = {
        "paths": [TRAIN / f"{img_id[0]}/{img_id}.npy" for img_id in val_df["id"].values],
        "labels": val_df[CLASSES].values.astype("f")
    }
    return train_path_label, val_path_label


def run_inference_loop(cfg, model, loader, device):
    model.to(device)
    model.eval()
    pred_list = []
    with torch.no_grad():
        for batch in tqdm(loader):
            x = to_device(batch["image"], device)
            y = model(x)
            pred_list.append(y.sigmoid().detach().cpu().numpy())
        
    pred_arr = np.concatenate(pred_list)
    del pred_list
    return pred_arr

## Run inference for each fold

In [ ]:
# # debug
# smpl_sub = smpl_sub.iloc[:32 * 10, :].copy()

In [ ]:
torch.backends.cudnn.benchmark = True
set_random_seed(1086, deterministic=True)
device = torch.device("cuda")

label_arr = train[CLASSES].values
oof_pred_arr = np.zeros((len(train), N_CLASSES))
test_pred_arr = np.zeros((N_FOLDS, len(smpl_sub), N_CLASSES))
score_list = []

test_path_label = {
    "paths": [DATA / f"test/{img_id[0]}/{img_id}.npy" for img_id in smpl_sub["id"].values],
    "labels": smpl_sub[CLASSES].values.astype("f")
}

In [ ]:
for fold_id in FOLDS:
    print(f"\n[fold {fold_id}]")
    tmp_dir = TRAINING_OUTPUT_PATH / f"fold{fold_id}"
    with open(tmp_dir / "config.yml", "r") as fr:
        cfg = Config(yaml.safe_load(fr), types=CONFIG_TYPES)
    val_idx = train.query("fold == @fold_id").index.values

    # # get_dataloader
    _, val_path_label = get_path_label(cfg, train)
    cfg["/dataset/val"].lazy_init(**val_path_label)
    cfg["/dataset/test"].lazy_init(**test_path_label)
    val_loader = cfg["/loader/val"]
    test_loader = cfg["/loader/test"]
    
    # # get model
    model_path = TRAINING_OUTPUT_PATH / f"best_metric_model_fold{fold_id}.pth"
    model = cfg["/model"]
    model.load_state_dict(torch.load(model_path, map_location=device))
    
    # # inference
    val_pred = run_inference_loop(cfg, model, val_loader, device)
    val_score = roc_auc_score(label_arr[val_idx], val_pred)
    oof_pred_arr[val_idx] = val_pred
    score_list.append([fold_id, val_score])
    
    test_pred_arr[fold_id] = run_inference_loop(cfg, model, test_loader, device)
    
    del cfg, val_idx, val_path_label
    del model, val_loader, test_loader
    torch.cuda.empty_cache()
    gc.collect()
    
    print(f"[fold {fold_id}] val score: {val_score:.4f}\n")

## Check oof score

In [ ]:
oof_score = roc_auc_score(label_arr, oof_pred_arr)
score_list.append(["oof", oof_score])
pd.DataFrame(score_list, columns=["fold", "metric"])

In [ ]:
oof_df = train.copy()
oof_df[CLASSES] = oof_pred_arr
oof_df.to_csv("./oof_prediction.csv", index=False)

## Make submission

In [ ]:
sub_df = smpl_sub.copy()
sub_df[CLASSES] = test_pred_arr.mean(axis=0)
sub_df.to_csv("./submission.csv", index=False)
sub_df.head()

# EOF